# Tabla players

In [37]:
import pandas as pd
import numpy as np

Para crear esta dimensión utilizaremos las ids de la tabla de hechos de player_stats, las cuales necesitaran un tratado porque vendrán duplicadas, debido a que en la tabla de hechos las player_id se repiten debido a la presencia de los mismos jugadores en varias temporadas.

In [38]:
players= pd.read_csv('player_final_merged_with_stats_sinTOT.csv')

In [39]:
players.head()

,primary_key_composed,player_id,player_name,position,age,team_abbreviation,GP,GS,MP,FG,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,season_id
0,1471_2021_MIA,1471,Precious Achiuwa,PF,21,MIA,28,2,14.6,2.6,...,1.3,2.7,4.0,0.6,0.4,0.5,1.0,1.9,6.5,2021
1,2_2021_MIL,2,Jaylen Adams,PG,24,MIL,6,0,2.8,0.2,...,0.0,0.5,0.5,0.3,0.0,0.0,0.0,0.2,0.3,2021
2,1274_2021_NOP,1274,Steven Adams,C,27,NOP,27,27,28.1,3.5,...,4.3,4.6,8.9,2.1,1.0,0.6,1.7,1.9,8.0,2021
3,1243_2021_MIA,1243,Bam Adebayo,C,23,MIA,26,26,33.6,7.4,...,1.9,7.3,9.2,5.3,1.0,1.0,3.0,2.6,19.9,2021
4,494_2021_SAS,494,LaMarcus Aldridge,C,35,SAS,18,18,26.7,5.9,...,0.8,3.5,4.3,1.9,0.4,0.9,0.9,1.5,14.1,2021


In [40]:
# Seleccionamos solo las columnas 2 y 3 (considerando que los índices empiezan en 0)
df_seleccionado = players.iloc[:, [1, 2]]

# Mostrar el DataFrame resultante
print(df_seleccionado)

      player_id        player_name
0          1471   Precious Achiuwa
1             2       Jaylen Adams
2          1274       Steven Adams
3          1243        Bam Adebayo
4           494  LaMarcus Aldridge
...         ...                ...
1666       1667      Justin Minaya
1667       1668       Kobi Simmons
1668       1669          Gabe York
1669       1670       RaiQuan Gray
1670       1671      Jacob Gilyard

[1671 rows x 2 columns]


Verificamos los nulos y duplicados:

In [41]:
# PORCENTAJE DE VALORES NULOS
nulos_total = df_seleccionado.isnull().sum()
porcentaje_nulos = (nulos_total / len(df_seleccionado)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

             Valores Nulos  Porcentaje Nulos
player_id                0               0.0
player_name              0               0.0


In [42]:
duplicados = df_seleccionado.duplicated(keep=False)

# Filtra las filas duplicadas
datos_duplicados = df_seleccionado[duplicados]
print(datos_duplicados)

      player_id               player_name
0          1471          Precious Achiuwa
2          1274              Steven Adams
3          1243               Bam Adebayo
4           494         LaMarcus Aldridge
6          1402  Nickeil Alexander-Walker
...         ...                       ...
1660       1661              Xavier Sneed
1661       1662         Lindell Wigginton
1662       1663            Jordan Schakel
1665       1666                Jay Scrubb
1668       1669                 Gabe York

[1427 rows x 2 columns]


In [43]:
# Eliminamos las filas duplicadas basadas en la columna 'player_id'
df_unique = df_seleccionado.drop_duplicates(subset='player_id', keep='first')

# Verificamos si las filas duplicadas fueron eliminadas
print(f"Cantidad de filas después de eliminar duplicados: {df_unique.shape[0]}")

Cantidad de filas después de eliminar duplicados: 766


Ahora se verificara que las ID esten presentes correctamente en la tabla de hechos y de dimensión, para asegurar que no falte ninguna.

In [44]:
# Obtener los player_id únicos de cada DataFrame
player_ids_df1 = set(players['player_id'])
player_ids_df2 = set(df_unique['player_id'])

# Encontrar player_ids que están en df1 pero no en df2
ids_solo_en_df1 = player_ids_df1 - player_ids_df2

# Encontrar player_ids que están en df2 pero no en df1
ids_solo_en_df2 = player_ids_df2 - player_ids_df1

# Verificar si hay diferencias
if not ids_solo_en_df1 and not ids_solo_en_df2:
    print("Todos los player_id están presentes en ambos DataFrames.")
else:
    if ids_solo_en_df1:
        print(f"player_id en df1 pero no en df2: {ids_solo_en_df1}")
    if ids_solo_en_df2:
        print(f"player_id en df2 pero no en df1: {ids_solo_en_df2}")

Todos los player_id están presentes en ambos DataFrames.


A este nuevo dataframe le tenemos que agregar la columna de Salarios, que la traemos desde un .csv externo de kaggle.

In [45]:
salario = pd.read_csv('Salarios.csv')

In [46]:
salario.head()

,Unnamed: 0,Player,Salary,Temp
0,0,Stephen Curry,48070014,2023
1,1,John Wall,47345760,2023
2,2,Russell Westbrook,47080179,2023
3,3,LeBron James,44474988,2023
4,4,Kevin Durant,44119845,2023


In [47]:
# Dropeamos las columnas en los índices 0 y 3
df_salario = salario.drop(salario.columns[[0, 3]], axis=1)

# Mostrar el DataFrame resultante
print(df_salario)

                Player    Salary
0        Stephen Curry  48070014
1            John Wall  47345760
2    Russell Westbrook  47080179
3         LeBron James  44474988
4         Kevin Durant  44119845
..                 ...       ...
462      Justin Minaya     35096
463       Kobi Simmons     32795
464          Gabe York     32171
465       RaiQuan Gray      5849
466      Jacob Gilyard      5849

[467 rows x 2 columns]


In [48]:
df_unique

,player_id,player_name
0,1471,Precious Achiuwa
1,2,Jaylen Adams
2,1274,Steven Adams
3,1243,Bam Adebayo
4,494,LaMarcus Aldridge
...,...,...
1664,1665,Jeenathan Williams
1666,1667,Justin Minaya
1667,1668,Kobi Simmons
1669,1670,RaiQuan Gray


In [49]:
# Renombrar la columna 'player_name' a 'Player' en df_unique para que coincida con df_salario
df_unique.rename(columns={'player_name': 'Player'}, inplace=True)

C:\Users\geram\AppData\Local\Temp\ipykernel_12164\3808535591.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique.rename(columns={'player_name': 'Player'}, inplace=True)


In [50]:
# Realiza el merge en base a la columna 'Player'
df_combined = pd.merge(df_unique, df_salario, on='Player', how='left')

In [51]:
# Rellenamos con 0 los datos de los jugadores que no sabemos su salario.
df_combined = df_combined.fillna(0)
df_combined['Salary'] = df_combined['Salary'].astype(int)

In [52]:
df_combined

,player_id,Player,Salary
0,1471,Precious Achiuwa,2840160
1,2,Jaylen Adams,0
2,1274,Steven Adams,17926829
3,1243,Bam Adebayo,30351780
4,494,LaMarcus Aldridge,0
...,...,...,...
761,1665,Jeenathan Williams,52644
762,1667,Justin Minaya,35096
763,1668,Kobi Simmons,32795
764,1670,RaiQuan Gray,5849


In [53]:
df_combined.dtypes

player_id     int64
Player       object
Salary        int64
dtype: object

Guardamos la tabla de dimensión player_info finalizada.

In [54]:
df_combined.to_csv('player_info_final.csv', index=False)